In [2]:
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from lstm import LSTM
from time_series_dataset import TimeSeriesDataset
from input_data import InputData
from trainer import Trainer
import pandas as pd
import plotly.express as px

In [3]:
train_df = pd.read_parquet("../../../data/time_series/train_df.parquet").asfreq("D")
validation_df = pd.read_parquet(
    "../../../data/time_series/validation_df.parquet"
).asfreq("D")
test_df = pd.read_parquet("../../../data/time_series/test_df.parquet").asfreq("D")

In [4]:
df = pd.concat([train_df, validation_df, test_df])

In [5]:
df.head()

,negative_count,positive_count,neutral_count,year,gap,gap_log
review_date,,,,,,
2001-11-01,297,2175,238,2001,1878,7.537963
2001-11-02,335,2076,226,2001,1741,7.462215
2001-11-03,266,1741,183,2001,1475,7.296413
2001-11-04,304,1890,191,2001,1586,7.36897
2001-11-05,366,2474,250,2001,2108,7.653495


In [6]:
scaler = MinMaxScaler()
sequence_length = 8
train_size_proportion = (len(train_df) + len(validation_df)) / len(df)

In [7]:
input_data = InputData(
    df=df,
    sequence_length=sequence_length,
    train_size_proportion=train_size_proportion,
    scaler=scaler,
)

In [8]:
X_train, X_test, y_train, y_test = input_data.create_training_and_test_sets()

In [9]:
# Hyperparameters
input_size = 1
hidden_size = 128
output_size = 1
learning_rate = 0.0001
num_epochs = 50
batch_size = 10

In [10]:
# Create data loaders
train_dataset = TimeSeriesDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize the model, loss function, and optimizer
model = LSTM(input_size, hidden_size, output_size, random_seed=13)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

trainer = Trainer(
    num_epochs=num_epochs,
    optimizer=optimizer,
    criterion=criterion,
    model=model,
    train_loader=train_loader,
    scaler=scaler,
)
trainer.train()

Epoch [1/50], Loss: 0.0018
Epoch [2/50], Loss: 0.0006
Epoch [3/50], Loss: 0.0003
Epoch [4/50], Loss: 0.0028
Epoch [5/50], Loss: 0.0
Epoch [6/50], Loss: 0.0004
Epoch [7/50], Loss: 0.0002
Epoch [8/50], Loss: 0.0025
Epoch [9/50], Loss: 0.0
Epoch [10/50], Loss: 0.0053
Epoch [11/50], Loss: 0.0263
Epoch [12/50], Loss: 0.0002
Epoch [13/50], Loss: 0.0001
Epoch [14/50], Loss: 0.0009
Epoch [15/50], Loss: 0.0014
Epoch [16/50], Loss: 0.0009
Epoch [17/50], Loss: 0.0
Epoch [18/50], Loss: 0.0001
Epoch [19/50], Loss: 0.0001
Epoch [20/50], Loss: 0.0014
Epoch [21/50], Loss: 0.0
Epoch [22/50], Loss: 0.0
Epoch [23/50], Loss: 0.0049
Epoch [24/50], Loss: 0.0009
Epoch [25/50], Loss: 0.0001
Epoch [26/50], Loss: 0.0008
Epoch [27/50], Loss: 0.0004
Epoch [28/50], Loss: 0.0
Epoch [29/50], Loss: 0.0
Epoch [30/50], Loss: 0.0
Epoch [31/50], Loss: 0.0
Epoch [32/50], Loss: 0.0001
Epoch [33/50], Loss: 0.0001
Epoch [34/50], Loss: 0.0013
Epoch [35/50], Loss: 0.0068
Epoch [36/50], Loss: 0.0001
Epoch [37/50], Loss: 0.0008


In [11]:
result = trainer.evaluate(X_test, y_test)
y_pred = result["y_pred"]
y_test = result["y_test"]

Root Mean Squared Error (RMSE): 11245.563249784384
Mean Absolute Error (MAE): 8877.97
Mean Absolute Percentage Error (MAPE): 11.06%


In [12]:
# Visualize predictions against actual data
df = input_data.df
train_size = input_data.train_size

In [19]:
data = {
    "Date": df.index[train_size + sequence_length :],
    "Valor real": y_test.flatten(),
    "Predicción": y_pred.flatten(),
}

# Convertir a DataFrame de pandas
plot_df = pd.DataFrame(data)

# Crear la figura usando Plotly Express
fig = px.line(
    plot_df,
    x="Date",
    y=["Valor real", "Predicción"],
    labels={"value": "Gap", "variable": "Legend"},
    title="Test vs Predicciones usando LSTM",
    markers=True,
)

# Mostrar la figura
fig.show()